In [93]:
# Essential imports
import boto3
import pandas as pd
import pymysql
import datetime as dt
import random

In [107]:
# Common Variables
port = 3306
user = 'admin'
password = 'password'


# General helper methods
def getDate():
    return dt.datetime.today().strftime('%Y-%m-%d') # YYYY-MM-DD

In [95]:
host1 = 'm1.cpx46c1uwiy7.us-east-1.rds.amazonaws.com' # Burner-m1
dbName1 = 'mDB1' # Don't think we can actually use this
host2 = 'a2-instance-1.cpx46c1uwiy7.us-east-1.rds.amazonaws.com' # Burner-a2
db1 = pymysql.connect(host = host1, user = user, password = password, database = dbName1, autocommit = True)
print('Connection 1 established?', db1)

Connection 1 established? <pymysql.connections.Connection object at 0x11a4d6160>


In [96]:
# Using the built-in DB that was established via Amazon RDS
cursor1 = db1.cursor()
q4 = ''' Use mDB1 '''
r4 = cursor1.execute(q4)
r4

0

In [97]:
# Establishing a connection with plain vanilla MySQL instance (create a new database)

db1 = pymysql.connect(host = host1, user = user, password = password, autocommit = True)
print('Connection 1 established?', db1)
cursor1 = db1.cursor()
r1 = None
try:
    q1 = ''' Create database db1 '''
    r1 = cursor1.execute(q1)
except Exception as e:
    print('Issue:', e)
    
q2 = ''' Drop database db1 '''
r2 = cursor1.execute(q2)

q3 = ''' Create database db1 '''
r3 = cursor1.execute(q3)

q4 = ''' Use db1'''
r4 = cursor1.execute(q4)

# Note: use Varchar for primary ke to avoid issue of arbitrarily long primary keys
q5 = '''
Create table Person (
ID VARCHAR(16),
firstName text,
lastName text,
primary key (ID)
)
'''

r5 = cursor1.execute(q5)

display(r1, r2, r3, r4, r5)

Connection 1 established? <pymysql.connections.Connection object at 0x11a4f1e20>
Issue: (1007, "Can't create database 'db1'; database exists")


None

2

1

0

0

In [98]:
# Get all the existing databases
cursor1.execute(''' SHOW DATABASES''')
display(cursor1.fetchall())

(('db1',),
 ('information_schema',),
 ('mDB1',),
 ('mysql',),
 ('performance_schema',))

In [99]:
# "Advanced" queries with the first database

q6 = ''' Show Tables ''' # Display all the tables in the DB
q7 = ''' Insert into Person values ('%s', '%s', '%s')''' % ('EEE833134', 'Zach', 'Zhu') # Add an entry into table
q8 = ''' Select * from Person ''' # 
q9 = ''' Desc Person '''
queries = [q6, q7, q8, q9]

for q in queries:
    try:
        cursor1.execute(q)
        display(cursor1.fetchall())
    except Exception as e:
        print('Issue:', e)


(('Person',),)

()

(('EEE833134', 'Zach', 'Zhu'),)

(('ID', 'varchar(16)', 'NO', 'PRI', None, ''),
 ('firstName', 'text', 'YES', '', None, ''),
 ('lastName', 'text', 'YES', '', None, ''))

In [100]:
# Convert MySQL database into pandas df--allowing for MANY more advanced querying
try:
    df1 = pd.read_sql('''SELECT * FROM Person''', con = db1)
    display(df1)
except Exception as e:
    print('error:', e)

,ID,firstName,lastName
0,EEE833134,Zach,Zhu


In [101]:
# Delete table
def deleteTable():
    try:
        q = ''' DROP Table QAResults '''
        cursor1.execute(q)
    except Exception as e:
        print('Error:', e)
#deleteTable()

In [102]:
# Helper/Utility Methods

# Create an initial table to contain QA Results
def createQATable():
    q = '''
        Create table if not exists QAResults(
            SerialNumber VARCHAR(30),
            BatchNumber VARCHAR(30),
            Date VARCHAR(10),
            Tester VARCHAR(20),
            FunctionalTest VARCHAR(4),
            LogMessage text,
            primary key (SerialNumber)
        )
    '''
    try:
        #cursor1.execute(q)
        display(executeQuery(q))
    except Exception as e:
        print('Issue with table creation:', e)
    
# Helper method to get an entry (based off of SN alone)
def getEntry(sn):
    try:
        q = ''' SELECT * FROM QAResults where SerialNumber = '{}' '''.format(sn)
        #cursor1.execute(q)
        display(executeQuery(q))
    except Exception as e:
        print('Issue with get entry:', e)

# Helper method to insert an entry
def insertEntry(sn, bn, ft, tst, msg, date = getDate()):
    try:
        q = ''' Insert into QAResults values ('%s', '%s', '%s', '%s', '%s', '%s')''' % (sn, bn, date, tst, ft, msg)
        display(executeQuery(q))
        #executeQuery(q)
    except Exception as e:
        print('Issue with add entry:', e)

# Delete an entry based on SN alone
def deleteEntry(sn):
    try:
        q = ''' Delete from QAResults where SerialNumber = '%s' ''' % (sn)
        #cursor1.execute(q)
        display(executeQuery(q))
    except Exception as e:
        print('Issue with delete entry:', e)

# Modify a field based off of SN
def modifyEntry(sn, field, newVal):
    try:
        q = ''' Update QAResults set {} = '{}' where SerialNumber = '{}' '''.format(field, newVal,sn)
        #cursor1.execute(q)
        display(executeQuery(q))
    except Exception as e:
        print('Issue with edit entry:', e)

# Get all the results for the table
def displayAll():
    try:
        print('\n\nQAResults Table\n')
        q = ''' Select * From QAResults '''
        #cursor1.execute(q)
        #display(cursor1.fetchall())
        display(executeQuery(q))
    except Exception as e:
        print('Issue with displaying table:', e)

def executeQuery(q):
    cursor1.execute(q) # Execute the query
    return cursor1.fetchall() # Display the output of said query

In [118]:
# Initial Testing the framework above
createQATable()
displayAll() # After initial table creation
insertEntry('EEE833134W0', 'E2', 'Zach', getDate(), 'Pass', 'Everything works')
insertEntry('EEE833134W1', 'E2', 'Ruohan', getDate(),'Pass', 'Everything works')
insertEntry('EEE833134W2', 'E2','Ryan', getDate(), 'Fail', 'Breakdown occurred')
displayAll() # After insertion of entries
modifyEntry('EEE833134W1', 'BatchNumber', 'F33')
displayAll() # After modification of an entry
deleteEntry('EEE833134W2')
displayAll() # After deletion of an entry

()



QAResults Table



(('28336714411', 'GG3', '2021-04-01', 'Ryan', 'Pass', 'Everything works'),
 ('28336714415', 'EE1', '2019-11-27', 'Zach', 'Pass', 'Everything works'),
 ('28336714419', 'FF2', '2020-05-02', 'Megan', 'Pass', 'Everything works'),
 ('28336714423', 'GG3', '2021-04-01', 'Nikki', 'Pass', 'Everything works'),
 ('28336714427', 'EE1', '2019-11-27', 'Ruohan', 'Pass', 'Everything works'),
 ('2833671443', 'EE1', '2019-11-27', 'Nikki', 'Pass', 'Everything works'),
 ('28336714431', 'FF2', '2020-05-02', 'Ryan', 'Pass', 'Everything works'),
 ('28336714435', 'GG3', '2021-04-01', 'Zach', 'Pass', 'Everything works'),
 ('28336714439', 'EE1', '2019-11-27', 'Megan', 'Pass', 'Everything works'),
 ('28336714443', 'FF2', '2020-05-02', 'Nikki', 'Pass', 'Everything works'),
 ('28336714447', 'GG3', '2021-04-01', 'Ruohan', 'Pass', 'Everything works'),
 ('2833671447', 'FF2', '2020-05-02', 'Ruohan', 'Pass', 'Everything works'),
 ('ABC221', 'FF2', '2020-05-02', 'Ryan', 'Pass', 'Everything works'),
 ('ABC2213', 'FF2', '

Issue with add entry: (1062, "Duplicate entry 'EEE833134W0' for key 'QAResults.PRIMARY'")
Issue with add entry: (1062, "Duplicate entry 'EEE833134W1' for key 'QAResults.PRIMARY'")


QAResults Table



(('28336714411', 'GG3', '2021-04-01', 'Ryan', 'Pass', 'Everything works'),
 ('28336714415', 'EE1', '2019-11-27', 'Zach', 'Pass', 'Everything works'),
 ('28336714419', 'FF2', '2020-05-02', 'Megan', 'Pass', 'Everything works'),
 ('28336714423', 'GG3', '2021-04-01', 'Nikki', 'Pass', 'Everything works'),
 ('28336714427', 'EE1', '2019-11-27', 'Ruohan', 'Pass', 'Everything works'),
 ('2833671443', 'EE1', '2019-11-27', 'Nikki', 'Pass', 'Everything works'),
 ('28336714431', 'FF2', '2020-05-02', 'Ryan', 'Pass', 'Everything works'),
 ('28336714435', 'GG3', '2021-04-01', 'Zach', 'Pass', 'Everything works'),
 ('28336714439', 'EE1', '2019-11-27', 'Megan', 'Pass', 'Everything works'),
 ('28336714443', 'FF2', '2020-05-02', 'Nikki', 'Pass', 'Everything works'),
 ('28336714447', 'GG3', '2021-04-01', 'Ruohan', 'Pass', 'Everything works'),
 ('2833671447', 'FF2', '2020-05-02', 'Ruohan', 'Pass', 'Everything works'),
 ('ABC221', 'FF2', '2020-05-02', 'Ryan', 'Pass', 'Everything works'),
 ('ABC2213', 'FF2', '

...



QAResults Table



(('28336714411', 'GG3', '2021-04-01', 'Ryan', 'Pass', 'Everything works'),
 ('28336714415', 'EE1', '2019-11-27', 'Zach', 'Pass', 'Everything works'),
 ('28336714419', 'FF2', '2020-05-02', 'Megan', 'Pass', 'Everything works'),
 ('28336714423', 'GG3', '2021-04-01', 'Nikki', 'Pass', 'Everything works'),
 ('28336714427', 'EE1', '2019-11-27', 'Ruohan', 'Pass', 'Everything works'),
 ('2833671443', 'EE1', '2019-11-27', 'Nikki', 'Pass', 'Everything works'),
 ('28336714431', 'FF2', '2020-05-02', 'Ryan', 'Pass', 'Everything works'),
 ('28336714435', 'GG3', '2021-04-01', 'Zach', 'Pass', 'Everything works'),
 ('28336714439', 'EE1', '2019-11-27', 'Megan', 'Pass', 'Everything works'),
 ('28336714443', 'FF2', '2020-05-02', 'Nikki', 'Pass', 'Everything works'),
 ('28336714447', 'GG3', '2021-04-01', 'Ruohan', 'Pass', 'Everything works'),
 ('2833671447', 'FF2', '2020-05-02', 'Ruohan', 'Pass', 'Everything works'),
 ('ABC221', 'FF2', '2020-05-02', 'Ryan', 'Pass', 'Everything works'),
 ('ABC2213', 'FF2', '

()



QAResults Table



(('28336714411', 'GG3', '2021-04-01', 'Ryan', 'Pass', 'Everything works'),
 ('28336714415', 'EE1', '2019-11-27', 'Zach', 'Pass', 'Everything works'),
 ('28336714419', 'FF2', '2020-05-02', 'Megan', 'Pass', 'Everything works'),
 ('28336714423', 'GG3', '2021-04-01', 'Nikki', 'Pass', 'Everything works'),
 ('28336714427', 'EE1', '2019-11-27', 'Ruohan', 'Pass', 'Everything works'),
 ('2833671443', 'EE1', '2019-11-27', 'Nikki', 'Pass', 'Everything works'),
 ('28336714431', 'FF2', '2020-05-02', 'Ryan', 'Pass', 'Everything works'),
 ('28336714435', 'GG3', '2021-04-01', 'Zach', 'Pass', 'Everything works'),
 ('28336714439', 'EE1', '2019-11-27', 'Megan', 'Pass', 'Everything works'),
 ('28336714443', 'FF2', '2020-05-02', 'Nikki', 'Pass', 'Everything works'),
 ('28336714447', 'GG3', '2021-04-01', 'Ruohan', 'Pass', 'Everything works'),
 ('2833671447', 'FF2', '2020-05-02', 'Ruohan', 'Pass', 'Everything works'),
 ('ABC221', 'FF2', '2020-05-02', 'Ryan', 'Pass', 'Everything works'),
 ('ABC2213', 'FF2', '

In [ ]:
# Functions + Variables to mass insert entries (consistently) into the database
snPrefix = ['EUHWQ3931-', 'ABC22', 'RRH43144', '283367144']
batchNums = ['EE1', 'FF2', 'GG3']
testers = ['Zach', 'Ryan', 'Ruohan', 'Nikki', 'Megan']
results = ['Pass', 'Fail']
dates = [getDate(), '2020-05-02', '2020-06-09', '2019-11-27', '2021-04-19', '2021-04-01']
messages = ['Camera issue', 'Night mode issue', 'Other issue-unlisted']
passMsg = 'Everything works'

def massInsertEntry(n):
    for i in range(n):
        sn = snPrefix[i % 4] + str(i)
        bn = batchNums[i % 3]
        t = testers[i % 5]
        res = results[int(i % 4 == 0)]
        d = dates[i % 6]
        msg = passMsg if res == 'Pass' else messages[i % 3]
        insertEntry(sn, bn, res, t, msg, d)

massInsertEntry(50)
displayAll()

In [119]:
# Convert into the tabular form... powerful
def convertToDF(tbl):
    try:
        df1 = pd.read_sql('''SELECT * FROM {} '''.format(tbl), con = db1)
        return df1
    except Exception as e:
        print('error:', e)

display(convertToDF('QAResults')) # This is so clean--I love it!

,SerialNumber,BatchNumber,Date,Tester,FunctionalTest,LogMessage
0,28336714411,GG3,2021-04-01,Ryan,Pass,Everything works
1,28336714415,EE1,2019-11-27,Zach,Pass,Everything works
2,28336714419,FF2,2020-05-02,Megan,Pass,Everything works
3,28336714423,GG3,2021-04-01,Nikki,Pass,Everything works
4,28336714427,EE1,2019-11-27,Ruohan,Pass,Everything works
5,2833671443,EE1,2019-11-27,Nikki,Pass,Everything works
6,28336714431,FF2,2020-05-02,Ryan,Pass,Everything works
7,28336714435,GG3,2021-04-01,Zach,Pass,Everything works
8,28336714439,EE1,2019-11-27,Megan,Pass,Everything works
9,28336714443,FF2,2020-05-02,Nikki,Pass,Everything works


In [125]:
# Date Queries

# Must obtain an exact match
def getDay(date):
    q = ''' Select * from QAResults where Date = '{}' '''.format(date)
    return executeQuery(q)

# Format: date is passed in as YYYY-MM to be matched against given dates
def getMonth(date):
    q = ''' Select * from QAResults where Substring(Date, 1, 7) = '{}' '''.format(date)
    return executeQuery(q)

# Get all entries for a given year
def getYear(year):
    q = ''' Select * from QAResults where Substring(Date, 1, 4) = '{}' '''.format(year)
    return executeQuery(q)

# Get all entries before a date (inclusive)
def getEntriesBefore(date):
    q = ''' Select * from QAResults where Date < '{}' '''.format(date)
    return executeQuery(q)
    
# Get all entries after a date (inclusive)
def getEntriesAfter(date):
    q = ''' Select * from QAResults where Date >= '{}' '''.format(date)
    return executeQuery(q)

# Get all the entries that fall b/w a specific start and end date
def getEntriesBetween(start, end):
    q = ''' Select * from QAResults where Date <= '{}' and Date >= '{}' '''.format(end, start)
    return executeQuery(q)

In [126]:
# Testing date framework
def testDateFunctions():
    day = '2021-04-19'
    month = '2021-06'
    year = '2019'
    startDay = '2021-04-19'
    endDay = '2021-06-10'
    print('\n\nGet entries on day:', day)
    display(getDay(day))
    print('\n\nGet entries on month:', month)
    display(getMonth(month))
    print('\n\nGet entries on year:', year)
    display(getYear(year))
    print('\n\nGet entries before day:', day)
    display(getEntriesBefore(day))
    print('\n\nGet entries after day:', day)
    display(getEntriesAfter(day))
    print('\n\nGet entries between:', startDay, 'and', endDay)
    display(getEntriesBetween(startDay, endDay))

testDateFunctions()



Get entries on day: 2021-04-19


(('EUHWQ3931-16', 'FF2', '2021-04-19', 'Ryan', 'Fail', 'Night mode issue'),
 ('EUHWQ3931-28', 'FF2', '2021-04-19', 'Nikki', 'Fail', 'Night mode issue'),
 ('EUHWQ3931-4', 'FF2', '2021-04-19', 'Megan', 'Fail', 'Night mode issue'),
 ('EUHWQ3931-40', 'FF2', '2021-04-19', 'Zach', 'Fail', 'Night mode issue'),
 ('RRH4314410', 'FF2', '2021-04-19', 'Zach', 'Pass', 'Everything works'),
 ('RRH4314422', 'FF2', '2021-04-19', 'Ruohan', 'Pass', 'Everything works'),
 ('RRH4314434', 'FF2', '2021-04-19', 'Megan', 'Pass', 'Everything works'),
 ('RRH4314446', 'FF2', '2021-04-19', 'Ryan', 'Pass', 'Everything works'))

None



Get entries on month: 2021-06


(('EUHWQ3931-0', 'EE1', '2021-06-10', 'Zach', 'Fail', 'Camera issue'),
 ('EUHWQ3931-12', 'EE1', '2021-06-10', 'Ruohan', 'Fail', 'Camera issue'),
 ('EUHWQ3931-24', 'EE1', '2021-06-10', 'Megan', 'Fail', 'Camera issue'),
 ('EUHWQ3931-36', 'EE1', '2021-06-10', 'Ryan', 'Fail', 'Camera issue'),
 ('EUHWQ3931-48', 'EE1', '2021-06-10', 'Nikki', 'Fail', 'Camera issue'),
 ('RRH4314418', 'EE1', '2021-06-10', 'Nikki', 'Pass', 'Everything works'),
 ('RRH4314430', 'EE1', '2021-06-10', 'Zach', 'Pass', 'Everything works'),
 ('RRH4314442', 'EE1', '2021-06-10', 'Ruohan', 'Pass', 'Everything works'),
 ('RRH431446', 'EE1', '2021-06-10', 'Ryan', 'Pass', 'Everything works'))

...

(('EEE833134W0', 'E2', 'Everything', 'Zach', '2021', 'Pass'),
 ('EEE833134W1', 'F33', 'Everything', 'Ruohan', '2021', 'Pass'),
 ('EUHWQ3931-0', 'EE1', '2021-06-10', 'Zach', 'Fail', 'Camera issue'),
 ('EUHWQ3931-12', 'EE1', '2021-06-10', 'Ruohan', 'Fail', 'Camera issue'),
 ('EUHWQ3931-16', 'FF2', '2021-04-19', 'Ryan', 'Fail', 'Night mode issue'),
 ('EUHWQ3931-24', 'EE1', '2021-06-10', 'Megan', 'Fail', 'Camera issue'),
 ('EUHWQ3931-28', 'FF2', '2021-04-19', 'Nikki', 'Fail', 'Night mode issue'),
 ('EUHWQ3931-36', 'EE1', '2021-06-10', 'Ryan', 'Fail', 'Camera issue'),
 ('EUHWQ3931-4', 'FF2', '2021-04-19', 'Megan', 'Fail', 'Night mode issue'),
 ('EUHWQ3931-40', 'FF2', '2021-04-19', 'Zach', 'Fail', 'Night mode issue'),
 ('EUHWQ3931-48', 'EE1', '2021-06-10', 'Nikki', 'Fail', 'Camera issue'),
 ('RRH4314410', 'FF2', '2021-04-19', 'Zach', 'Pass', 'Everything works'),
 ('RRH4314418', 'EE1', '2021-06-10', 'Nikki', 'Pass', 'Everything works'),
 ('RRH4314422', 'FF2', '2021-04-19', 'Ruohan', 'Pass', '

None



Get entries between: 2021-04-19 and 2021-06-10


(('EUHWQ3931-0', 'EE1', '2021-06-10', 'Zach', 'Fail', 'Camera issue'),
 ('EUHWQ3931-12', 'EE1', '2021-06-10', 'Ruohan', 'Fail', 'Camera issue'),
 ('EUHWQ3931-16', 'FF2', '2021-04-19', 'Ryan', 'Fail', 'Night mode issue'),
 ('EUHWQ3931-24', 'EE1', '2021-06-10', 'Megan', 'Fail', 'Camera issue'),
 ('EUHWQ3931-28', 'FF2', '2021-04-19', 'Nikki', 'Fail', 'Night mode issue'),
 ('EUHWQ3931-36', 'EE1', '2021-06-10', 'Ryan', 'Fail', 'Camera issue'),
 ('EUHWQ3931-4', 'FF2', '2021-04-19', 'Megan', 'Fail', 'Night mode issue'),
 ('EUHWQ3931-40', 'FF2', '2021-04-19', 'Zach', 'Fail', 'Night mode issue'),
 ('EUHWQ3931-48', 'EE1', '2021-06-10', 'Nikki', 'Fail', 'Camera issue'),
 ('RRH4314410', 'FF2', '2021-04-19', 'Zach', 'Pass', 'Everything works'),
 ('RRH4314418', 'EE1', '2021-06-10', 'Nikki', 'Pass', 'Everything works'),
 ('RRH4314422', 'FF2', '2021-04-19', 'Ruohan', 'Pass', 'Everything works'),
 ('RRH4314430', 'EE1', '2021-06-10', 'Zach', 'Pass', 'Everything works'),
 ('RRH4314434', 'FF2', '2021-04-1

None

In [120]:
def testQueries():
    queries = []
    q1 = ''' SELECT * FROM QAResults where FunctionalTest = 'Pass' ''' # Get all passing results
    q2 = ''' Select * From QAResults where Date = '2021-06-10' ''' # Get specific date
    queries += [q1, q2]
    for q in queries:
        try:
            print('NEW QUERY\n\n\n')
            cursor1.execute(q)
            display(cursor1.fetchall())
        except Exception as e:
            print('Error in querying:', e)
testQueries()

NEW QUERY





(('28336714411', 'GG3', '2021-04-01', 'Ryan', 'Pass', 'Everything works'),
 ('28336714415', 'EE1', '2019-11-27', 'Zach', 'Pass', 'Everything works'),
 ('28336714419', 'FF2', '2020-05-02', 'Megan', 'Pass', 'Everything works'),
 ('28336714423', 'GG3', '2021-04-01', 'Nikki', 'Pass', 'Everything works'),
 ('28336714427', 'EE1', '2019-11-27', 'Ruohan', 'Pass', 'Everything works'),
 ('2833671443', 'EE1', '2019-11-27', 'Nikki', 'Pass', 'Everything works'),
 ('28336714431', 'FF2', '2020-05-02', 'Ryan', 'Pass', 'Everything works'),
 ('28336714435', 'GG3', '2021-04-01', 'Zach', 'Pass', 'Everything works'),
 ('28336714439', 'EE1', '2019-11-27', 'Megan', 'Pass', 'Everything works'),
 ('28336714443', 'FF2', '2020-05-02', 'Nikki', 'Pass', 'Everything works'),
 ('28336714447', 'GG3', '2021-04-01', 'Ruohan', 'Pass', 'Everything works'),
 ('2833671447', 'FF2', '2020-05-02', 'Ruohan', 'Pass', 'Everything works'),
 ('ABC221', 'FF2', '2020-05-02', 'Ryan', 'Pass', 'Everything works'),
 ('ABC2213', 'FF2', '

NEW QUERY





(('EUHWQ3931-0', 'EE1', '2021-06-10', 'Zach', 'Fail', 'Camera issue'),
 ('EUHWQ3931-12', 'EE1', '2021-06-10', 'Ruohan', 'Fail', 'Camera issue'),
 ('EUHWQ3931-24', 'EE1', '2021-06-10', 'Megan', 'Fail', 'Camera issue'),
 ('EUHWQ3931-36', 'EE1', '2021-06-10', 'Ryan', 'Fail', 'Camera issue'),
 ('EUHWQ3931-48', 'EE1', '2021-06-10', 'Nikki', 'Fail', 'Camera issue'),
 ('RRH4314418', 'EE1', '2021-06-10', 'Nikki', 'Pass', 'Everything works'),
 ('RRH4314430', 'EE1', '2021-06-10', 'Zach', 'Pass', 'Everything works'),
 ('RRH4314442', 'EE1', '2021-06-10', 'Ruohan', 'Pass', 'Everything works'),
 ('RRH431446', 'EE1', '2021-06-10', 'Ryan', 'Pass', 'Everything works'))

In [121]:
# Establishing a connection with Amazon Aurora (with pre-existing DB)
host2 = 'a2-instance-1.cpx46c1uwiy7.us-east-1.rds.amazonaws.com'
db2 = pymysql.connect(host = host2, user = user, password = password, database = 'aDB2', autocommit = True)
print('Connection 2 established?', db2)
cursor2 = db2.cursor()
try:
    qa0 = ''' Use aDB2 '''
    cursor2.execute(qa0)
except Exception as e:
    print('Issue:', e)



Connection 2 established? <pymysql.connections.Connection object at 0x11a504f70>


In [122]:
# Create a new database--Amazon Aurora Edition
r02 = None
try:
    q02 = ''' Create database db2 '''
    r02 = cursor2.execute(q02)
except Exception as e:
    print('Issue:', e)

q03 = ''' Use db2 '''
r03 = cursor2.execute(q03)
display(q03, r03)

' Use db2 '

0